In [1]:
%run dados.ipynb

In [2]:
spark = (SparkSession.builder
        .master("local")
        .appName("projetosoulcode")
        .config('spark.jars','gcs-connector-hadoop3-latest.jar' )
        .config('spark.hadoop.fs.gs.impl','com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem')
        .config('spark.hadoop.fs.gs.auth.service.account.enable', 'true')
        .config("spark.hadoop.google.cloud.auth.service.account.json.keyfile",'soulcodeproject-44ce61622773.json')
        .config('spark.hadoop.fs.gs.project.id','soulcodeproject')
        .getOrCreate())
spark.conf.set("spark.sql.caseSensitive", "true")

In [3]:
schema = (
    StructType([
        StructField("ID",IntegerType(), False),
        StructField("AnoNascimento", IntegerType(), False),
        StructField("GrauInstrução", StringType(), True),
        StructField("EstadoCivil", StringType(), True),
        StructField("Renda", DoubleType(), True),
        StructField("NumCrianca", IntegerType(), True),
        StructField("NumAdolescente", IntegerType(), True),
        StructField("DataConta", StringType(), True),
        StructField("UltimaCompra", IntegerType(), True),
        StructField("QtdVinho", IntegerType(), True),
        StructField("QtdFrutas", IntegerType(), True),
        StructField("QtdCarne", IntegerType(), True),
        StructField("QtdPeixe", IntegerType(), True),
        StructField("QtdDoces", IntegerType(), True),
        StructField("QtdOuro", IntegerType(), True),
        StructField("QtdComprasDesconto", IntegerType(), True),
        StructField("QtdComprasWeb", IntegerType(), True),
        StructField("QtdCompraCatalogo", IntegerType(), True),
        StructField("QtdCompraLoja", IntegerType(), True),
        StructField("NumVisitasSite", IntegerType(), True),
        StructField("AceitouCam3", IntegerType(), True),
        StructField("AceitouCam4", IntegerType(), True),
        StructField("AceitouCam5", IntegerType(), True),
        StructField("AceitouCam1", IntegerType(), True),
        StructField("AceitouCam2", IntegerType(), True),
        StructField("Reclamacao", IntegerType(), True),
        StructField("CustoContato", IntegerType(), True),
        StructField("Receita", IntegerType(), True),
        StructField("UltimaCampanha", IntegerType(), True),
    ])
)

In [4]:
df2= (
    spark
       .read
       .format("csv")
       .option("header", "true")
       .option("delimiter", ",")
       .load('gs://soulcodeprojeto/marketing_campaign.csv',schema=schema)
)

In [5]:
df2.show()
print(df2.count(), len(df2.columns))

+----+-------------+-------------+-----------+-------+----------+--------------+----------+------------+--------+---------+--------+--------+--------+-------+------------------+-------------+-----------------+-------------+--------------+-----------+-----------+-----------+-----------+-----------+----------+------------+-------+--------------+
|  ID|AnoNascimento|GrauInstrução|EstadoCivil|  Renda|NumCrianca|NumAdolescente| DataConta|UltimaCompra|QtdVinho|QtdFrutas|QtdCarne|QtdPeixe|QtdDoces|QtdOuro|QtdComprasDesconto|QtdComprasWeb|QtdCompraCatalogo|QtdCompraLoja|NumVisitasSite|AceitouCam3|AceitouCam4|AceitouCam5|AceitouCam1|AceitouCam2|Reclamacao|CustoContato|Receita|UltimaCampanha|
+----+-------------+-------------+-----------+-------+----------+--------------+----------+------------+--------+---------+--------+--------+--------+-------+------------------+-------------+-----------------+-------------+--------------+-----------+-----------+-----------+-----------+-----------+----------

In [6]:
a=[x[0] for x in df2.select('GrauInstrução').distinct().collect()]
df2=df2.replace(a, ['Segundo Ciclo', 'PhD', 'Mestrado', 'Graduado', 'Básico'])
df2.show()

+----+-------------+-------------+-----------+-------+----------+--------------+----------+------------+--------+---------+--------+--------+--------+-------+------------------+-------------+-----------------+-------------+--------------+-----------+-----------+-----------+-----------+-----------+----------+------------+-------+--------------+
|  ID|AnoNascimento|GrauInstrução|EstadoCivil|  Renda|NumCrianca|NumAdolescente| DataConta|UltimaCompra|QtdVinho|QtdFrutas|QtdCarne|QtdPeixe|QtdDoces|QtdOuro|QtdComprasDesconto|QtdComprasWeb|QtdCompraCatalogo|QtdCompraLoja|NumVisitasSite|AceitouCam3|AceitouCam4|AceitouCam5|AceitouCam1|AceitouCam2|Reclamacao|CustoContato|Receita|UltimaCampanha|
+----+-------------+-------------+-----------+-------+----------+--------------+----------+------------+--------+---------+--------+--------+--------+-------+------------------+-------------+-----------------+-------------+--------------+-----------+-----------+-----------+-----------+-----------+----------

In [7]:
b=[x[0] for x in df2.select('EstadoCivil').distinct().collect()]
df2=df2.replace(b,['Só se vive uma vez', 'Juntado', 'Casado', 'Absurdo', 'Viúvo','Divorciado', 'Sozinho', 'Solteiro'])
df2.show()

+----+-------------+-------------+-----------+-------+----------+--------------+----------+------------+--------+---------+--------+--------+--------+-------+------------------+-------------+-----------------+-------------+--------------+-----------+-----------+-----------+-----------+-----------+----------+------------+-------+--------------+
|  ID|AnoNascimento|GrauInstrução|EstadoCivil|  Renda|NumCrianca|NumAdolescente| DataConta|UltimaCompra|QtdVinho|QtdFrutas|QtdCarne|QtdPeixe|QtdDoces|QtdOuro|QtdComprasDesconto|QtdComprasWeb|QtdCompraCatalogo|QtdCompraLoja|NumVisitasSite|AceitouCam3|AceitouCam4|AceitouCam5|AceitouCam1|AceitouCam2|Reclamacao|CustoContato|Receita|UltimaCampanha|
+----+-------------+-------------+-----------+-------+----------+--------------+----------+------------+--------+---------+--------+--------+--------+-------+------------------+-------------+-----------------+-------------+--------------+-----------+-----------+-----------+-----------+-----------+----------

In [8]:
df2.select([count(when(isnan(c), c)).alias(c) for c in df2.columns]).show()

+---+-------------+-------------+-----------+-----+----------+--------------+---------+------------+--------+---------+--------+--------+--------+-------+------------------+-------------+-----------------+-------------+--------------+-----------+-----------+-----------+-----------+-----------+----------+------------+-------+--------------+
| ID|AnoNascimento|GrauInstrução|EstadoCivil|Renda|NumCrianca|NumAdolescente|DataConta|UltimaCompra|QtdVinho|QtdFrutas|QtdCarne|QtdPeixe|QtdDoces|QtdOuro|QtdComprasDesconto|QtdComprasWeb|QtdCompraCatalogo|QtdCompraLoja|NumVisitasSite|AceitouCam3|AceitouCam4|AceitouCam5|AceitouCam1|AceitouCam2|Reclamacao|CustoContato|Receita|UltimaCampanha|
+---+-------------+-------------+-----------+-----+----------+--------------+---------+------------+--------+---------+--------+--------+--------+-------+------------------+-------------+-----------------+-------------+--------------+-----------+-----------+-----------+-----------+-----------+----------+-----------

In [9]:
df2.select([count(when(col(c).isNull(), c)).alias(c) for c in df2.columns]).show()

+---+-------------+-------------+-----------+-----+----------+--------------+---------+------------+--------+---------+--------+--------+--------+-------+------------------+-------------+-----------------+-------------+--------------+-----------+-----------+-----------+-----------+-----------+----------+------------+-------+--------------+
| ID|AnoNascimento|GrauInstrução|EstadoCivil|Renda|NumCrianca|NumAdolescente|DataConta|UltimaCompra|QtdVinho|QtdFrutas|QtdCarne|QtdPeixe|QtdDoces|QtdOuro|QtdComprasDesconto|QtdComprasWeb|QtdCompraCatalogo|QtdCompraLoja|NumVisitasSite|AceitouCam3|AceitouCam4|AceitouCam5|AceitouCam1|AceitouCam2|Reclamacao|CustoContato|Receita|UltimaCampanha|
+---+-------------+-------------+-----------+-----+----------+--------------+---------+------------+--------+---------+--------+--------+--------+-------+------------------+-------------+-----------------+-------------+--------------+-----------+-----------+-----------+-----------+-----------+----------+-----------

In [10]:
df2=df2.dropna()

In [11]:
df2.describe().show()

+-------+----------------+------------------+-------------+-----------+------------------+------------------+------------------+----------+-----------------+------------------+------------------+------------------+-----------------+------------------+-----------------+------------------+------------------+------------------+-----------------+-----------------+-------------------+------------------+-------------------+-------------------+--------------------+-------------------+------------+-------+-------------------+
|summary|              ID|     AnoNascimento|GrauInstrução|EstadoCivil|             Renda|        NumCrianca|    NumAdolescente| DataConta|     UltimaCompra|          QtdVinho|         QtdFrutas|          QtdCarne|         QtdPeixe|          QtdDoces|          QtdOuro|QtdComprasDesconto|     QtdComprasWeb| QtdCompraCatalogo|    QtdCompraLoja|   NumVisitasSite|        AceitouCam3|       AceitouCam4|        AceitouCam5|        AceitouCam1|         AceitouCam2|         Recla

In [12]:
df2=df2.drop('Receita', 'CustoContato')

In [13]:
df2.summary().show()

+-------+----------------+------------------+-------------+-----------+------------------+------------------+------------------+----------+-----------------+------------------+------------------+------------------+-----------------+------------------+-----------------+------------------+------------------+------------------+-----------------+-----------------+-------------------+------------------+-------------------+-------------------+--------------------+-------------------+-------------------+
|summary|              ID|     AnoNascimento|GrauInstrução|EstadoCivil|             Renda|        NumCrianca|    NumAdolescente| DataConta|     UltimaCompra|          QtdVinho|         QtdFrutas|          QtdCarne|         QtdPeixe|          QtdDoces|          QtdOuro|QtdComprasDesconto|     QtdComprasWeb| QtdCompraCatalogo|    QtdCompraLoja|   NumVisitasSite|        AceitouCam3|       AceitouCam4|        AceitouCam5|        AceitouCam1|         AceitouCam2|         Reclamacao|     UltimaCamp

In [20]:
print(df2.count(), len(df2.columns))

2216 29


In [15]:
df2=df2.withColumnRenamed('NumCrianca','QtdCriancas').withColumnRenamed('NumAdolescente', 'QtdAdolescentes')
df2.show()

+----+-------------+-------------+-----------+-------+-----------+---------------+----------+------------+--------+---------+--------+--------+--------+-------+------------------+-------------+-----------------+-------------+--------------+-----------+-----------+-----------+-----------+-----------+----------+--------------+
|  ID|AnoNascimento|GrauInstrução|EstadoCivil|  Renda|QtdCriancas|QtdAdolescentes| DataConta|UltimaCompra|QtdVinho|QtdFrutas|QtdCarne|QtdPeixe|QtdDoces|QtdOuro|QtdComprasDesconto|QtdComprasWeb|QtdCompraCatalogo|QtdCompraLoja|NumVisitasSite|AceitouCam3|AceitouCam4|AceitouCam5|AceitouCam1|AceitouCam2|Reclamacao|UltimaCampanha|
+----+-------------+-------------+-----------+-------+-----------+---------------+----------+------------+--------+---------+--------+--------+--------+-------+------------------+-------------+-----------------+-------------+--------------+-----------+-----------+-----------+-----------+-----------+----------+--------------+
|5524|         1957

In [16]:
df2=df2.withColumn('TotalGasto', F.col('QtdVinho')+F.col('QtdFrutas')+F.col('QtdCarne')+F.col('QtdPeixe')+F.col('QtdOuro')).orderBy('TotalGasto', ascending=False)

In [17]:
df2=df2.withColumn('Filhos', F.col('QtdCriancas')+F.col('QtdAdolescentes')).orderBy('Filhos', ascending=False)

In [28]:
df2.groupBy("EstadoCivil").agg({'QtdVinho':'mean','QtdDoces':'mean' }).show()

+------------------+------------------+------------------+
|       EstadoCivil|     avg(QtdVinho)|     avg(QtdDoces)|
+------------------+------------------+------------------+
|             Viúvo|367.13157894736844| 37.86842105263158|
|        Divorciado| 324.8448275862069| 26.81896551724138|
|           Sozinho|184.66666666666666|               7.0|
|          Solteiro| 291.3312101910828|27.072186836518046|
|           Absurdo|             355.5|              30.5|
|Só se vive uma vez|             322.0|               3.0|
|            Casado|299.85530921820305|26.751458576429403|
|           Juntado|308.40314136125653|26.232111692844676|
+------------------+------------------+------------------+



In [29]:
df2.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- AnoNascimento: integer (nullable = true)
 |-- GrauInstrução: string (nullable = true)
 |-- EstadoCivil: string (nullable = true)
 |-- Renda: double (nullable = true)
 |-- QtdCriancas: integer (nullable = true)
 |-- QtdAdolescentes: integer (nullable = true)
 |-- DataConta: string (nullable = true)
 |-- UltimaCompra: integer (nullable = true)
 |-- QtdVinho: integer (nullable = true)
 |-- QtdFrutas: integer (nullable = true)
 |-- QtdCarne: integer (nullable = true)
 |-- QtdPeixe: integer (nullable = true)
 |-- QtdDoces: integer (nullable = true)
 |-- QtdOuro: integer (nullable = true)
 |-- QtdComprasDesconto: integer (nullable = true)
 |-- QtdComprasWeb: integer (nullable = true)
 |-- QtdCompraCatalogo: integer (nullable = true)
 |-- QtdCompraLoja: integer (nullable = true)
 |-- NumVisitasSite: integer (nullable = true)
 |-- AceitouCam3: integer (nullable = true)
 |-- AceitouCam4: integer (nullable = true)
 |-- AceitouCam5: integer (nullable =

In [32]:
df3 = (df2.withColumn("Status", F.when((F.col('Filhos') >0) & (F.col('EstadoCivil') == 'Solteiro') |(F.col('EstadoCivil') == 'Sozinho') |(F.col('EstadoCivil') == 'Viúvo') , F.lit("Solteiro com Filhos"))
                      .otherwise(F.lit('Casado com Filhos'))))

In [33]:
df3.show()

+-----+-------------+-------------+-----------+-------+-----------+---------------+----------+------------+--------+---------+--------+--------+--------+-------+------------------+-------------+-----------------+-------------+--------------+-----------+-----------+-----------+-----------+-----------+----------+--------------+----------+------+-------------------+
|   ID|AnoNascimento|GrauInstrução|EstadoCivil|  Renda|QtdCriancas|QtdAdolescentes| DataConta|UltimaCompra|QtdVinho|QtdFrutas|QtdCarne|QtdPeixe|QtdDoces|QtdOuro|QtdComprasDesconto|QtdComprasWeb|QtdCompraCatalogo|QtdCompraLoja|NumVisitasSite|AceitouCam3|AceitouCam4|AceitouCam5|AceitouCam1|AceitouCam2|Reclamacao|UltimaCampanha|TotalGasto|Filhos|             Status|
+-----+-------------+-------------+-----------+-------+-----------+---------------+----------+------------+--------+---------+--------+--------+--------+-------+------------------+-------------+-----------------+-------------+--------------+-----------+-----------+---